<a href="https://colab.research.google.com/github/JayTiptown/self-consistency-ensemble/blob/main/self_consistency_ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# imports

import asyncio
import random
from collections import defaultdict
import nest_asyncio
nest_asyncio.apply()  # Needed to run asyncio in Colab
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import pandas as pd
from google.colab import userdata

In [ ]:
# model import

!pip install openai
!pip install anthropic
!pip install google-genai

import openai
import anthropic
import google.genai

openai.api_key = userdata.get("OPENAI_API_KEY")
anthropic.api_key = userdata.get("ANTHROPIC_API_KEY")
google.genai.api_key = userdata.get("GEMINI_API_KEY")

In [ ]:
# simulated chain of thought sample

async def sample_chain(prompt, model_spec):

  provider, model = model_spec

  full_prompt = prompt + "\nLet's think step by step."

  if provider == 'openai':

    # Run the blocking call in a thread so we don't block asyncio
    # Use openai.chat.completions.create for chat models like gpt-4
    response = await asyncio.to_thread(
        openai.chat.completions.create,  # Changed to chat.completions.create
        model="gpt-4",     # or any completions model
        messages=[{"role": "user", "content": full_prompt}],  # Added messages parameter
        temperature=0.7,
        max_tokens=150,
        # logprobs=1
    )

    choice = response.choices[0]
    trace = choice.message.content  # Access content from message
    # crude extraction: last line is the answer
    answer = trace.strip().split("\n")[-1]

    # logprobs is not supported in chat completions as of now
    # token_lps = choice.logprobs.token_logprobs or []
    # total_logprob = sum(token_lps)

  # fill these out for relevant providers

  elif provider == 'anthropic':
    pass
  elif provider == 'gemini':
    pass
  else:
    raise ValueError(f"Unknown provider: {provider}")

  total_logprob = 0 # Placeholder logprobs isn't directly supported for chat models.
  return answer, total_logprob

In [ ]:
# adaptive controller
def controller(prompt, max_budget=5):
  """
  Enforce a fixed sampling budget.
  """
  return max_budget

In [ ]:
# parallel orchestrator

async def orchestrate_samples(prompt, budget, model_spec):
  tasks = [
    asyncio.create_task(sample_chain(prompt, model_spec))
    for _ in range(budget)
  ]
  return await asyncio.gather(*tasks)

In [ ]:
# aggregator

def aggregate_votes(samples):
  """
  samples: list of (answer_str, log_prob_float)
  Returns the answer with highest total log_prob.
  """
  score_dict = defaultdict(float)
  for answer, log_prob in samples:
      score_dict[answer] += log_prob
  winner = max(score_dict.items(), key=lambda x: x[1])[0]
  return winner, dict(score_dict)

In [ ]:
# self-consistency run

def run_self_consistency(prompt, model_spec, max_budget):
  budget = controller(prompt, max_budget)
  samples = asyncio.run(orchestrate_samples(prompt, budget, model_spec))
  winner, scores = aggregate_votes(samples)
  print(f"Prompt: {prompt}")
  print(f"Samples and log_probs: {samples}")
  print(f"Aggregated scores: {scores}")
  print(f"Winner: {winner}")
  return winner, scores

run_self_consistency("Who invented the atomic bomb?", max_budget=20)

Prompt: Who invented the atomic bomb?
Samples and log_probs: [('3. The actual creation of the atomic bomb was the result of the Manhattan Project, a U.S. government research project during World War II. The project was', 0), ('4. The work on atomic bomb started with the discovery of nuclear fission by German scientists Otto Hahn and Fritz Strassmann in 1938, and its theoretical explanation by Lise Meitner and Otto Fr', 0), ('4', 0), ('4. This led to the establishment of the Manhattan Project, a secret U', 0), ('4. The actual development of the atomic bomb', 0), ('So, it can be said', 0), ('So, while Oppenheimer played a leading role in the physical invention of the atomic bomb', 0), ('4. Therefore, while many people', 0), ('4. In summary, while it was a group effort, J. Robert Oppenheimer is often credited as the "father of the atomic bomb" due to his role as the director of', 0), ('6. The first successful test of the atomic bomb, codenamed "Trinity", was conducted on July 16', 0), ('4

('3. The actual creation of the atomic bomb was the result of the Manhattan Project, a U.S. government research project during World War II. The project was',
 {'3. The actual creation of the atomic bomb was the result of the Manhattan Project, a U.S. government research project during World War II. The project was': 0.0,
  '4. The work on atomic bomb started with the discovery of nuclear fission by German scientists Otto Hahn and Fritz Strassmann in 1938, and its theoretical explanation by Lise Meitner and Otto Fr': 0.0,
  '4': 0.0,
  '4. This led to the establishment of the Manhattan Project, a secret U': 0.0,
  '4. The actual development of the atomic bomb': 0.0,
  'So, it can be said': 0.0,
  'So, while Oppenheimer played a leading role in the physical invention of the atomic bomb': 0.0,
  '4. Therefore, while many people': 0.0,
  '4. In summary, while it was a group effort, J. Robert Oppenheimer is often credited as the "father of the atomic bomb" due to his role as the director o